In [1]:
import numpy as np
import pandas as pd

In [22]:
from sklearn.impute import SimpleImputer  # its fill the null values in the that particular column
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [3]:
df = pd.read_csv('covid_toy.csv')

In [4]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_teset = train_test_split(df.drop('has_covid',axis=1), 
                                                     df['has_covid'],
                                                     test_size=0.2)

In [6]:
df['cough'].value_counts()

cough
Mild      62
Strong    38
Name: count, dtype: int64

In [7]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

### Column Transformer

In [8]:
from sklearn.compose import ColumnTransformer

In [24]:
transformer = ColumnTransformer(transformers=[
    
    ('tnf1',  SimpleImputer(), ['fever']),
    
    ('tnf2',  OrdinalEncoder(categories=[['Mild','Strong']]), ['cough']),
    
    ('tnf3',  OneHotEncoder(sparse_output=False, drop='first'),  ['gender','city']),
    
], remainder='passthrough')

In [35]:
transformer

ColumnTransformer(remainder='passthrough',
                  transformers=[('tnf1', SimpleImputer(), ['fever']),
                                ('tnf2',
                                 OrdinalEncoder(categories=[['Mild',
                                                             'Strong']]),
                                 ['cough']),
                                ('tnf3',
                                 OneHotEncoder(drop='first',
                                               sparse_output=False),
                                 ['gender', 'city'])])

In [25]:
tf = transformer.fit_transform(X_train)
tf

array([[104.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   1.        ,  44.        ],
       [101.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  83.        ],
       [104.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  17.        ],
       [ 98.        ,   1.        ,   0.        ,   0.        ,
          0.        ,   1.        ,  69.        ],
       [ 99.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   1.        ,  14.        ],
       [102.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   1.        ,  74.        ],
       [101.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   1.        ,  65.        ],
       [101.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,   8.        ],
       [103.        ,   0.        ,   1.        ,   0.        ,
          1.    

In [26]:
transformer.fit_transform(X_train).shape

(80, 7)

In [27]:
transformer.fit_transform(X_test).shape

(20, 7)

In [28]:
pd.DataFrame(tf)

,0,1,2,3,4,5,6
0,104.000000,0.0,1.0,0.0,0.0,1.0,44.0
1,101.000000,0.0,0.0,0.0,1.0,0.0,83.0
2,104.000000,0.0,0.0,0.0,1.0,0.0,17.0
3,98.000000,1.0,0.0,0.0,0.0,1.0,69.0
4,99.000000,0.0,0.0,0.0,0.0,1.0,14.0
...,...,...,...,...,...,...,...
75,103.000000,1.0,1.0,0.0,1.0,0.0,70.0
76,104.000000,0.0,0.0,0.0,0.0,0.0,18.0
77,100.680556,1.0,0.0,0.0,0.0,1.0,20.0
78,100.000000,0.0,1.0,0.0,0.0,0.0,11.0


### 1. Simple

In [29]:
# adding simple imputer to fever column
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])

# also the test data
X_test_fever = si.fit_transform(X_test[['fever']])

X_train_fever.shape

(80, 1)

In [30]:
# Ordinal Encoding -> cough
oe = OrdinalEncoder(categories= [['Mild','Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])

# also test the data
X_test_cough = oe.fit_transform(X_test[['cough']])

X_train_cough.shape

(80, 1)

In [31]:
# OneHotEncoding -> gender, city
ohe = OneHotEncoder(drop='first', sparse_output=False)
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])

# also the test data
X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])

X_train_gender_city.shape

(80, 4)

In [32]:
# Extracting Age
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

# also the test data
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values

X_train_age.shape

(80, 1)

In [33]:
# train data concatenate
X_train_transformed = np.concatenate((X_train_age, X_train_fever, X_train_gender_city, X_train_cough), axis=1)

# test data concatenate
X_test_transformed = np.concatenate((X_test_age, X_test_fever, X_test_gender_city, X_test_cough), axis=1)

In [34]:
X_train_transformed.shape

(80, 7)